# MODIS Access and Analysis
#### Perry Oddo <perry.oddo@nasa.gov>

### Part 1: Download MODIS with Python

In [ ]:
# Import required packages
import fiona
import rasterio
import xarray as xr
import rioxarray as rxr
import earthpy.plot as ep

from osgeo import gdal
from pathlib import Path
from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler
from modis_tools.auth import add_earthdata_netrc, remove_earthdata_netrc

We first need to authenticate our Python session using login information from `earthdata.nasa.gov`

To register with NASA Earthdata, see information [here](https://urs.earthdata.nasa.gov/)

In [2]:
# Enter NASA Earthdata credentials
username = "###########"
password = "###########"

# Authenticate MODIS session with login information
session = ModisSession(username=username, password=password)

Select the desired MODIS data product. Here, we're downloading the [MOD13A1](https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/MOD13A1) product which provides a Vegetation Index (VI) value at a per pixel basis (500-m resolution):

In [3]:
# Query the MODIS catalog for collections
collection_client = CollectionApi(session=session)
collections = collection_client.query(short_name="MOD13A1", version="006")

# Query the selected collection for granules
granule_client = GranuleApi.from_collection(collections[0], session=session)

Next, set up parameters for your data download:
<div class="alert alert-block alert-warning"> <b>NOTE: </b> Be careful with large date ranges. MODIS files can be large (<50 mb) so use small date range to test</div>

In [ ]:
# Set target dates
start = "2022-10-01"
end = "2022-10-14"

# Find bounding box of La Plata River Basin
plata_shape = "Data/cuenca_del_plata_shp.shp"

with fiona.open(plata_shape) as box:
    plata_bbox = box.bounds

print("Bounding box for La Plata:", plata_bbox)

# Send Query
plata_granules = granule_client.query(start_date=start, end_date=end, bounding_box=plata_bbox)

In [5]:
# Download the granules that overlap this basin (uncomment to download)
GranuleHandler.download_from_granules(plata_granules, session, path="Data/")

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 9/9 [05:43<00:00, 38.17s/file]


[WindowsPath('Data/MOD13A1.A2022273.h13v10.006.2022297221228.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h11v11.006.2022297221612.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h13v12.006.2022297221428.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h14v11.006.2022297221624.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h13v11.006.2022297221420.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h12v12.006.2022297222855.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h12v10.006.2022297222131.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h12v11.006.2022297222528.hdf'),
 WindowsPath('Data/MOD13A1.A2022273.h11v10.006.2022297223830.hdf')]

The MOD13A1 files for this period should download to your `Data` folder.

To start working with them, let's create a list of all downloaded MODIS files:

In [ ]:
# Set path to Data folder
data_path = Path("Data")

# Find all MODIS HDF files
mod_files = data_path.glob("*.hdf")
mod_files = [f for f in mod_files]

print("Number of MODIS files:", str(len(mod_files)))

### Part 2: Manipulating MODIS data

Open one of the MODIS files to see the different bands it contains:

In [ ]:
# User rasterio package to open the first file
# Print all the bands to see subdatasets
with rasterio.open(mod_files[0]) as test_file:
    for band in test_file.subdatasets:
        print(band)

Extract the **16 day NDVI** band from the MODIS files:

In [ ]:
# Create empty list to store data
ndvi_files = []

# Loop through MODIS files
for file in mod_files:
    
    # Open MODIS file as rasterio object and save to temporary variable
    ndvi_single = rxr.open_rasterio(file,
                                   masked=True,
                                   variable="500m 16 days NDVI").squeeze()
    
    # Add variable to list
    ndvi_files.append(ndvi_single)
    
# Explore sample output
print(ndvi_files[0])

In [ ]:
# Try plotting single file with earthpy package:
title = "MOD13A1 16-day NDVI"
ep.plot_bands(ndvi_files[0]["500m 16 days NDVI"], cmap="RdYlGn", cols=1, title=title)

### Part 3: Combining and Plotting NDVI data

Notice that the scale doesn't appear correct in our plot -- NDVI values should fall between **[-1,1]**

Let's merge the separate files and apply a scaling factor of `0.0001`

In [10]:
# Merge files into single raster with xarray package
merged = xr.merge(ndvi_files)

# Apply scaling factor
scaled = merged * 0.0001

# Save merged object as geotiff
scaled.rio.to_raster("Outputs/NDVI_merged.tif")

In [11]:
# Reproject merged geotiff to correct coordinate reference system (epsg:4326)
warp_path_in = "Outputs/NDVI_merged.tif"
warp_path_out = "Outputs/NDVI_warp.tif"

# Run gdalwarp utility through Python API
ds = gdal.Warp(warp_path_out, warp_path_in, dstSRS='EPSG:4326')
ds = None

We have merged and reprojected our NDVI raster. Now we want to clip it to our study area (La Plata Basin shapefile)

In [12]:
import rasterio.mask

# Define inputs and outputs
clip_path_in = "Outputs/NDVI_warp.tif"
clip_path_out = "Outputs/NDVI_clip.tif"

# Open shapefile to use as mask
with fiona.open(plata_shape, "r") as shapefile:
    mask = [feature["geometry"] for feature in shapefile]

# Clip raster with mask
with rasterio.open(clip_path_in) as src:
    out_image, out_transform = rasterio.mask.mask(src, mask, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

# Save to file
with rasterio.open(clip_path_out, "w", **out_meta) as dest:
    dest.write(out_image)

Plot clipped NDVI map

In [13]:
# Open clipped NDVI with rioxarray
new = rxr.open_rasterio("Outputs/NDVI_clip.tif")

In [ ]:
# Create plot
%matplotlib inline

new.plot(cmap="RdYlGn")

### Part 4: Adapting this script

Now it's **your turn**. 

Make a copy of this jupyter notebook (e.g. `MODIS-Access-new.ipynb`) so you can explore the data yourselves.

Things to try:
* First, change the credentials (cell [2]) and try authenticating with your own username / password
* Change some parameters for the data query (e.g. new start/end dates or [new bounding box](http://bboxfinder.com/)
* Try exploring a different band (e.g. try plotting EVI instead of NDVI)

### Part 5: Explore a new MODIS product

This tutorial specifically uses the **MOD13A1** (Vegetation Indices 16-Day L3 Global 500m) product. Try exploring a different MODIS product from the [Data Products Table](https://modis.gsfc.nasa.gov/data/dataprod/). 

<div class="alert alert-block alert-info"> <b>Discussion: </b> Share which product you selected and discuss a possible application area</div>